# Many To Many RNN for Language Modeling
Previously I used LLM to help with writin code, but here everything is developed by me!<br>
As for dataset I used "Silent Dan" by Ozon671games3 (short description of this audiobook: Ce Pizda)

In [24]:
import torch
from torch import nn
from torch.nn import functional as F
import random

In [25]:
device = "cuda"

In [26]:
text = open("data.txt", "r").read()

chars = sorted(set(text))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

In [49]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.ih1 = nn.Linear(input_size + hidden_size, hidden_size)
    self.dropout = nn.Dropout(0.1)
    self.relu = nn.ReLU()
    self.h1h2 = nn.Linear(hidden_size * 2, hidden_size)
    self.dropout = nn.Dropout(0.1)
    self.relu = nn.ReLU()
    self.h2h3 = nn.Linear(hidden_size * 2, hidden_size)
    self.h3o = nn.Linear(hidden_size, output_size)

  def init_hidden(self, device):
    return [torch.zeros((1, self.hidden_size), device=device) for _ in range(3)]

  def forward(self, input, hidden):
    h1, h2, h3 = hidden
    cat_inp = torch.cat((input, h1), dim=1)
    h1_ = self.relu(self.dropout(self.ih1(cat_inp)))

    h1h2 = torch.cat((h1_, h2), dim=1)
    h2_ = self.relu(self.dropout(self.h1h2(h1h2)))

    h2h3 = torch.cat((h2_, h3), dim=1)
    h3_ = self.relu(self.dropout(self.h2h3(h2h3)))

    o = self.h3o(h3_)

    return o, (h1_, h2_, h3_)

In [97]:
input_size = output_size = vocab_size
hidden_size = 32

rnn = RNN(input_size, hidden_size, output_size).to(device)
optim = torch.optim.Adam(rnn.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [98]:
def generate(start="", len=10):
  chars = [stoi[ch] for ch in start]
  h1, h2, h3 = rnn.init_hidden(device)
  for i in range(len):
    input = F.one_hot(torch.tensor([chars[-1]]), vocab_size).to(device)
    logits, (h1, h2, h3) = rnn(input, (h1, h2, h3))
    probs = torch.softmax(logits, dim=1)
    ix = torch.multinomial(probs, 1).item()
    chars.append(ix)
  return "".join([itos[ch] for ch in chars])

In [99]:
from tqdm import tqdm

In [100]:
seq_len = 16
epochs = 50
block_per_epoch = 3_000

for epoch in range(epochs):
  start_idx = torch.randint(0, len(text)-block_per_epoch, (1,)).item()
  for i in tqdm(range(start_idx, start_idx+block_per_epoch)):
    chunk = text[i:i+seq_len]
    target = text[i+1:i+seq_len+1]
    input = [stoi[ch] for ch in chunk]
    target = torch.tensor([stoi[ch] for ch in target]).to(device)
    input = F.one_hot(torch.tensor(input), vocab_size).to(device)

    h1, h2, h3 = rnn.init_hidden(device)
    loss_chunk = 0
    for x in range(len(input)):
      inp = input[x].unsqueeze(0)
      logits, (h1, h2, h3) = rnn(inp, (h1, h2, h3))
      tg = target[x].unsqueeze(0)
      loss = loss_fn(logits, tg)
      loss_chunk += loss
    optim.zero_grad()
    loss_chunk.backward()
    optim.step()
  if (epoch+1) % 10 == 0:
    print(f"Loss: {loss_chunk.item()}")
    print(generate(random.choice(chars), 10))

100%|██████████| 3000/3000 [01:04<00:00, 46.41it/s]


Loss: 46.317596435546875
котя. нерал


100%|██████████| 3000/3000 [01:05<00:00, 45.95it/s]


Loss: 35.73217010498047
Эд, лань-тр


100%|██████████| 3000/3000 [01:04<00:00, 46.57it/s]


Loss: 45.22844696044922
вешишь! Пру


100%|██████████| 3000/3000 [01:04<00:00, 46.58it/s]


Loss: 45.88179016113281
Я нувась тя


100%|██████████| 3000/3000 [01:04<00:00, 46.65it/s]

Loss: 35.8166618347168
Вокохсялж?



In [112]:
print(generate("Дэн ", 1000))

Дэн я  нину.
отямелась, ко, и Да, я отдес под ссдободевил т грою дамая в ной тейгей И зашнацо с что Я ве.
Я выш сВыварава приби мне ист менашинет.
«тдеди.
Ннончарна? Васкией.
претол подвекесь вы коонам зомофия выбесле. по всё подкяникоющака вамало баваннот, и уприл зевкоки, .
М ому, у мней
Мы, в зде, казко?.
е савнитовом, зди так эедало над короашавал, каки нешомо вызавиня
я в про. оботозром. Квно осты ю, с вашажиячий дувем.
Дошимскавюй всё, вва ки креза ежибо ко вомо обузалаити приетно...
Качемсяю подё проя дежьсьхчя дне пол Даловивтцея, педывесяе, прапраЯ цивинланинячит ожок отобел, вовенивнушю я ди рамкоене за вако, нри на посчто, это Тевко стубороверазоли! Чтлом.
Я издот нипал котезо сказяюхно чтойю. Я сили имкоесста, скокия ма пмачего.
Я нежу меня.
Доненижатаз и, каминте-тавеебя приноек.
«Луизи.
вы.
Кежко о, воячиуголожоонуха было ке?
Зотьжигошужинай в с ук и это я я не с чтронготловны! немик -па за.
Но и премть потсолачна. отнамали?
ешири долок чтотити азйзнелях но что  вмашросе.

# Okay, I can believe ozon671games3 could write something like this

In [113]:
torch.save(rnn.state_dict(), "/content/params.pth")